In [1]:
!suricata -v

Suricata 6.0.15
USAGE: suricata [OPTIONS] [BPF FILTER]

	-c <path>                            : path to configuration file
	-T                                   : test configuration file (use with -c)
	-i <dev or ip>                       : run in pcap live mode
	-F <bpf filter file>                 : bpf filter file
	-r <path>                            : run in pcap file/offline mode
	-q <qid[:qid]>                       : run in inline nfqueue mode (use colon to specify a range of queues)
	-s <path>                            : path to signature file loaded in addition to suricata.yaml settings (optional)
	-S <path>                            : path to signature file loaded exclusively (optional)
	-l <dir>                             : default log directory
	-D                                   : run as daemon
	-k [all|none]                        : force checksum check (all) or disabled it (none)
	-V                                   : display Suricata version
	-v                  

In [2]:
!suricata-update enable-source tgreen/hunting

22/11/2023 -- 10:48:42 - <Info> -- Using data-directory /var/lib/suricata.
22/11/2023 -- 10:48:42 - <Info> -- Using Suricata configuration /etc/suricata/suricata.yaml
22/11/2023 -- 10:48:42 - <Info> -- Using /usr/share/suricata/rules for Suricata provided rules.
22/11/2023 -- 10:48:42 - <Info> -- Found Suricata version 6.0.15 at /usr/sbin/suricata.
22/11/2023 -- 10:48:42 - <Info> -- Creating directory /var/lib/suricata/update/sources
22/11/2023 -- 10:48:42 - <Info> -- Enabling default source et/open
22/11/2023 -- 10:48:42 - <Info> -- Source tgreen/hunting enabled


In [3]:
!tree /var/lib/suricata

/var/lib/suricata
└── update
    ├── cache
    │   └── index.yaml
    └── sources
        ├── et-open.yaml
        └── tgreen-hunting.yaml

3 directories, 3 files


In [4]:
!tree /usr/share/suricata

/usr/share/suricata
└── rules
    ├── app-layer-events.rules
    ├── decoder-events.rules
    ├── dhcp-events.rules
    ├── dnp3-events.rules
    ├── dns-events.rules
    ├── files.rules
    ├── http2-events.rules
    ├── http-events.rules
    ├── ipsec-events.rules
    ├── kerberos-events.rules
    ├── modbus-events.rules
    ├── mqtt-events.rules
    ├── nfs-events.rules
    ├── ntp-events.rules
    ├── rfb-events.rules
    ├── smb-events.rules
    ├── smtp-events.rules
    ├── ssh-events.rules
    ├── stream-events.rules
    └── tls-events.rules

1 directory, 20 files


In [9]:
LOGDIR = "/tmp/logs"

In [19]:
!rm -rf $LOGDIR && mkdir $LOGDIR

In [2]:
URL = "https://www.malware-traffic-analysis.net/2022/01/03/2022-01-01-thru-03-server-activity-with-log4j-attempts.pcap.zip"

In [34]:
import urllib.request

In [3]:
OUTPUT = "/tmp/malware-pcap.zip"

In [36]:
urllib.request.urlretrieve(URL, OUTPUT)

('/tmp/malware-pcap.zip', <http.client.HTTPMessage at 0x7f72d35407f0>)

In [37]:
!ls -lh /tmp/ | grep pcap

-rw-r--r--. 1 tanmwhitw tanmwhitw 1.3M Nov 22 11:27 malware-pcap.zip


In [1]:
from zipfile import ZipFile

In [3]:
with ZipFile(OUTPUT, "r") as zip:
    zip.extractall(path="/tmp", pwd="infected".encode("utf-8"))

In [5]:
import glob

In [6]:
FILES = glob.glob("/tmp/*.pcap")
FILES

['/tmp/2022-01-01-thru-03-server-activity-with-log4j-attempts.pcap']

In [7]:
PCAP = FILES[0]

In [20]:
!suricata -S /usr/share/suricata/rules/http-events.rules -l $LOGDIR -r $PCAP -v

22/11/2023 -- 21:22:43 - <Notice> - This is Suricata version 6.0.15 RELEASE running in USER mode
22/11/2023 -- 21:22:43 - <Info> - CPUs/cores online: 4
22/11/2023 -- 21:22:43 - <Info> - Setting engine mode to IDS mode by default
22/11/2023 -- 21:22:43 - <Info> - master exception-policy set to: auto


22/11/2023 -- 21:22:43 - <Info> - fast output device (regular) initialized: fast.log
22/11/2023 -- 21:22:43 - <Info> - eve-log output device (regular) initialized: eve.json
22/11/2023 -- 21:22:43 - <Info> - stats output device (regular) initialized: stats.log
22/11/2023 -- 21:22:43 - <Info> - 1 rule files processed. 49 rules successfully loaded, 0 rules failed
22/11/2023 -- 21:22:43 - <Info> - Threshold config parsed: 0 rule(s) found
22/11/2023 -- 21:22:43 - <Info> - 49 signatures processed. 0 are IP-only rules, 0 are inspecting packet payload, 49 inspect application layer, 0 are decoder event only
22/11/2023 -- 21:22:43 - <Notice> - all 5 packet processing threads, 4 management threads initialized, engine started.
22/11/2023 -- 21:22:43 - <Info> - Starting file run for /tmp/2022-01-01-thru-03-server-activity-with-log4j-attempts.pcap
22/11/2023 -- 21:22:43 - <Info> - No packets with invalid checksum, assuming checksum offloading is NOT used
22/11/2023 -- 21:22:44 - <Info> - pcap file /

In [16]:
import pandas
import json

In [15]:
pandas.options.display.html.use_mathjax = False

In [50]:
with open("/tmp/logs/eve.json", "r") as handle:
    DATAFRAME = pandas.json_normalize([
        json.loads(line) for line in handle
    ])
for col in ['pcap_cnt', 'src_port', 'dest_port']:
    DATAFRAME[col] = (
        DATAFRAME[col]
        .fillna(0)
        .astype(int)
    )
DATAFRAME['timestamp'] = pandas.to_datetime(DATAFRAME['timestamp'])
DATAFRAME[['pcap_cnt', 
           'timestamp', 
           'proto', 
           'event_type', 
           'src_ip', 
           'src_port', 
           'dest_ip', 
           'dest_port']].sort_values('timestamp').set_index('pcap_cnt')

,timestamp,proto,event_type,src_ip,src_port,dest_ip,dest_port
pcap_cnt,,,,,,,
0,2022-01-01 03:00:13.076985+03:00,TCP,flow,188.208.210.62,58185,198.71.247.91,8080
0,2022-01-01 03:00:13.076985+03:00,UDP,flow,146.88.240.4,49229,198.71.247.91,19
0,2022-01-01 03:00:13.076985+03:00,TCP,flow,47.93.245.108,50456,198.71.247.91,2377
0,2022-01-01 03:00:13.076985+03:00,TCP,flow,45.146.165.16,51392,198.71.247.91,3310
0,2022-01-01 03:00:13.076985+03:00,TCP,flow,89.248.165.53,59923,198.71.247.91,8803
...,...,...,...,...,...,...,...
39188,2022-01-03 22:10:13.587035+03:00,TCP,anomaly,198.71.247.91,80,157.245.70.127,59520
39191,2022-01-03 22:10:13.737337+03:00,TCP,alert,157.245.70.127,59520,198.71.247.91,80
39191,2022-01-03 22:10:13.737337+03:00,TCP,http,157.245.70.127,59520,198.71.247.91,80


In [52]:
DATAFRAME.sort_values('timestamp').set_index('pcap_cnt').event_type == 'alert'

pcap_cnt
0        False
0        False
0        False
0        False
0        False
         ...  
39188    False
39191     True
39191    False
39193    False
0        False
Name: event_type, Length: 25756, dtype: bool

In [46]:
DATAFRAME_ALERT = (
    DATAFRAME
    .loc[DATAFRAME.event_type == 'alert']
    .sort_values('timestamp')
    .dropna(how='all',axis=1)
)

In [48]:
DATAFRAME_ALERT[['pcap_cnt',
                 'timestamp',  
                 'alert.signature', 
                 'alert.category', 
                 'alert.severity', 
                 'proto',
                 'event_type', 
                 'src_ip', 
                 'src_port', 
                 'dest_ip', 
                 'dest_port']].sort_values('timestamp').set_index('pcap_cnt')

,timestamp,alert.signature,alert.category,alert.severity,proto,event_type,src_ip,src_port,dest_ip,dest_port
pcap_cnt,,,,,,,,,,
12795,2022-01-02 02:31:06.846992+03:00,SURICATA HTTP URI terminated by non-compliant ...,Generic Protocol Command Decode,3.0,TCP,alert,66.240.205.34,37244,198.71.247.91,80
12795,2022-01-02 02:31:06.846992+03:00,SURICATA HTTP METHOD terminated by non-complia...,Generic Protocol Command Decode,3.0,TCP,alert,66.240.205.34,37244,198.71.247.91,80
14739,2022-01-02 05:47:27.450922+03:00,SURICATA HTTP missing Host header,Generic Protocol Command Decode,3.0,TCP,alert,157.245.70.127,47092,198.71.247.91,80
14747,2022-01-02 05:47:27.740869+03:00,SURICATA HTTP missing Host header,Generic Protocol Command Decode,3.0,TCP,alert,157.245.70.127,47546,198.71.247.91,80
29250,2022-01-03 07:05:25.122456+03:00,SURICATA HTTP request field missing colon,Generic Protocol Command Decode,3.0,TCP,alert,147.182.237.4,58372,198.71.247.91,80
29250,2022-01-03 07:05:25.122456+03:00,SURICATA HTTP Host header invalid,Generic Protocol Command Decode,3.0,TCP,alert,147.182.237.4,58372,198.71.247.91,80
29250,2022-01-03 07:05:25.122456+03:00,SURICATA HTTP Unexpected Request body,Generic Protocol Command Decode,3.0,TCP,alert,147.182.237.4,58372,198.71.247.91,80
39181,2022-01-03 22:10:13.436562+03:00,SURICATA HTTP missing Host header,Generic Protocol Command Decode,3.0,TCP,alert,157.245.70.127,58982,198.71.247.91,80
39191,2022-01-03 22:10:13.737337+03:00,SURICATA HTTP missing Host header,Generic Protocol Command Decode,3.0,TCP,alert,157.245.70.127,59520,198.71.247.91,80
